In [3]:
import pandas as pd
import wandb

api = wandb.Api()
entity, project = "tpn", "rllib1"
runs = api.runs(entity + "/" + project)
ALGORITHM_TYPES = ["independent", "centralized", "shared"]
REWARD_TYPES = ["type_1", "type_2", "type_3"]
ALGORITHM_TYPE = "independent"
REWARD_TYPE = "type_1"

In [6]:
_run_stats = dict()
collated_performances_df = pd.DataFrame(
    [],
    columns = ['step_penalty', 'algorithm_type', 'reward_type',
        'episode_length', 'episode_reward', 'epsiode_assists',\
               'eval_length', 'eval_reward', 'eval_assists']
).fillna(0)


for run in runs:
    # if run._state != "finished":
    if run._state != "finished":
        if run._state == "running":
            _run_stats['running_runs'] = _run_stats.get('running_runs', 0) + 1
        else:
            _run_stats['crashed_runs'] = _run_stats.get('crashed_runs', 0) + 1
        continue
    _run_stats['finished_runs'] = _run_stats.get('finished_runs', 0) + 1

    config = run.config
    algo_type = config['algorithm_type']
    reward_type = config['env_config']['reward_type']
    step_penalty = config['env_config']['step_penalty']
    collated_performances_df.loc[len(collated_performances_df)] = [
        step_penalty,
        algo_type,
        reward_type,
        run.summary['episode_len_mean'],
        run.summary['episode_reward_mean'],
        run.summary['episode_assists_mean'],
        run.summary['eval_episode_len'],
        run.summary['eval_reward'],
        run.summary['eval_assists']
    ]

print(f"F: {_run_stats.get('finished_runs', 0)}, R: {_run_stats.get('running_runs', 0)}, C: {_run_stats.get('crashed_runs', 0)}")
collated_performances_df = collated_performances_df.groupby(['step_penalty', 'reward_type', 'algorithm_type']).agg(['mean', 'std', 'count'])

F: 410, R: 0, C: 98


In [7]:
collated_performances_df


episode_length                   \
                                                  mean        std count   
step_penalty reward_type algorithm_type                                   
-0.03        type_1      centralized         72.019444  12.211697    18   
                         independent         64.681818   7.077238    22   
                         shared              66.691304   7.995981    23   
             type_2      centralized         74.967222   9.578594    18   
                         independent         68.006364   6.727602    22   
                         shared              70.750000   6.300526    23   
-0.01        type_1      centralized         65.571818  10.635292    22   
                         independent         57.720417   4.907540    24   
                         shared              63.690417   5.532977    24   
             type_2      centralized         64.170526   9.840471    19   
                         independent         59.150870   6.354029    23   
                         shared              67.367200   4.809649    25   
 0.00        type_1      centralized         60.765000   9.945730    24   
                         independent         55.835833   5.624127    24   
                         shared              62.115000   7.232060    26   
             type_2      centralized         63.604783  12.551065    23   
                         independent         55.552000   6.071008    25   
                         shared              63.480000   4.852674    25   

                                        episode_reward                  \
                                                  mean       std count   
step_penalty reward_type algorithm_type                                  
-0.03        type_1      centralized          9.659950  0.414153    18   
                         independent          9.449150  0.303938    22   
                         shared               9.399513  0.209230    23   
             type_2      centralized          9.949856  0.289244    18   
                         independent          9.818323  0.270215    22   
                         shared              10.148413  0.331568    23   
-0.01        type_1      centralized          6.762282  0.661987    22   
                         independent          6.973433  0.044150    24   
                         shared               6.888083  0.089319    24   
             type_2      centralized          7.123642  0.180096    19   
                         independent          7.216922  0.087969    23   
                         shared               7.430004  0.134209    25   
 0.00        type_1      centralized          5.762500  0.218319    24   
                         independent          5.878333  0.115445    24   
                         shared               5.694231  0.292112    26   
             type_2      centralized          5.881739  0.473704    23   
                         independent          6.163840  0.171087    25   
                         shared               6.317520  0.195893    25   

                                        epsiode_assists                  \
                                                   mean       std count   
step_penalty reward_type algorithm_type                                   
-0.03        type_1      centralized           0.332222  0.151096    18   
                         independent           0.367727  0.153031    22   
                         shared                0.530870  0.152819    23   
             type_2      centralized           0.362778  0.170149    18   
                         independent           0.426818  0.156886    22   
                         shared                0.905217  0.336450    23   
-0.01        type_1      centralized           0.364545  0.150578    22   
                         independent           0.357500  0.097189    24   
                         shared                0.445000  0.100823    24   
         

In [9]:
print(collated_performances_df.to_latex(
    float_format="%.2f",
    sparsify=True,
    longtable=True,
))

\begin{longtable}{llrrrrrrrrrrrrrrrrrr}
\toprule
 &  & \multicolumn{3}{r}{episode_length} & \multicolumn{3}{r}{episode_reward} & \multicolumn{3}{r}{epsiode_assists} & \multicolumn{3}{r}{eval_length} & \multicolumn{3}{r}{eval_reward} & \multicolumn{3}{r}{eval_assists} \\
 &  & mean & std & count & mean & std & count & mean & std & count & mean & std & count & mean & std & count & mean & std & count \\
reward_type & algorithm_type &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\endfirsthead
\toprule
 &  & \multicolumn{3}{r}{episode_length} & \multicolumn{3}{r}{episode_reward} & \multicolumn{3}{r}{epsiode_assists} & \multicolumn{3}{r}{eval_length} & \multicolumn{3}{r}{eval_reward} & \multicolumn{3}{r}{eval_assists} \\
 &  & mean & std & count & mean & std & count & mean & std & count & mean & std & count & mean & std & count & mean & std & count \\
reward_type & algorithm_type &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\endhead
\midrule
\multicolu

In [10]:
_run_stats = dict()
collated_df = None
for run in runs:
    # if run._state != "finished":
    if run._state != "finished":
        if run._state == "running":
            _run_stats['running_runs'] = _run_stats.get('running', 0) + 1
        else:
            _run_stats['crashed_runs'] = _run_stats.get('crashed', 0) + 1
        continue
    _run_stats['finished_runs'] = _run_stats.get('finished', 0) + 1

    config = run.config
    if config['algorithm_type'] == ALGORITHM_TYPE and \
        config['env_config']['reward_type'] == REWARD_TYPE:
        _run_stats['matching_runs'] = _run_stats.get('matching_runs', 0) + 1

        for file in run.files():
            if 'analysis' in file.name:
                analysis_table = pd.read_json(
                    run.file(run.summary['analysis_table']['path']).download(exist_ok=True), 
                    orient='split')
                # Add extra fields and columns here 
                # Training Summary - (last logged values)
                analysis_table['training_iteration'] = run.summary['training_iteration']
                analysis_table['episodes_total'] = run.summary['episodes_total']
                analysis_table['episode_reward_mean'] = run.summary['episode_reward_mean']
                analysis_table['episode_len_mean'] = run.summary['episode_len_mean']
                analysis_table['episode_assists_mean'] = run.summary['episode_assists_mean']
                analysis_table['eval_episode_len'] = run.summary['eval_episode_len']
                analysis_table['eval_rewards'] = run.summary['eval_reward']
                analysis_table['eval_assists'] = run.summary['eval_assists']

                analysis_table.attrs['run_ids_count'] = analysis_table.attrs.get('run_ids_count', 0) + 1
                analysis_table.attrs['algorithm_type'] = config['algorithm_type']
                analysis_table.attrs['reward_type'] = config['env_config']['reward_type']
                if collated_df is None:
                    collated_df = analysis_table
                else:
                    collated_df = pd.concat([collated_df, analysis_table])

print(f"Matched runs: {_run_stats.get('matching_runs', 0)}, F: {_run_stats.get('finished_runs', 0)}, R: {_run_stats.get('running_runs', 0)}, C: {_run_stats.get('crashed_runs', 0)}")
print(f"{ALGORITHM_TYPE}, {REWARD_TYPE}")
print(collated_df.shape)

Matched runs: 2, F: 1, R: 0, C: 0
independent, type_1
(48, 23)


In [13]:
grouped_by_runs = collated_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension'])
grouped_by_runs[[str(i) for i in range(100, 1000, 100)]].agg(['mean', 'std', 'count'])

100                \
                                                           mean           std   
mode     agent_a    agent_b    test     dimension                               
modified predator_0 predator_1 ccm      x          5.102474e-01  1.739593e-01   
                                        y          3.262265e-01  2.015810e-01   
                               ccm_pval x          1.288860e-06  1.822724e-06   
                                        y          1.273575e-01  1.801108e-01   
                               granger  x          0.000000e+00  0.000000e+00   
                                        y          0.000000e+00  0.000000e+00   
         predator_1 predator_0 ccm      x          5.825572e-01  1.637663e-01   
                                        y          4.208703e-01  1.906459e-02   
                               ccm_pval x          1.575514e-22  2.228113e-22   
                                        y          6.220892e-02  8.797669e-02   
                               granger  x          0.000000e+00  0.000000e+00   
                                        y          0.000000e+00  0.000000e+00   
original predator_0 predator_1 ccm      x          7.392823e-01  5.375988e-02   
                                        y          4.305419e-01  8.519938e-02   
                               ccm_pval x          1.854663e-11  2.613816e-11   
                                        y          1.058315e-01  1.419689e-01   
                               granger  x          0.000000e+00  0.000000e+00   
                                        y          0.000000e+00  0.000000e+00   
         predator_1 predator_0 ccm      x          6.873071e-01  2.571562e-01   
                                        y          6.959656e-01  2.030641e-01   
                               ccm_pval x          5.018213e-02  7.096824e-02   
                                        y          1.260795e-05  1.783034e-05   
                               granger  x          0.000000e+00  0.000000e+00   
                                        y          0.000000e+00  0.000000e+00   

                                                                  200  \
                                                  count          mean   
mode     agent_a    agent_b    test     dimension                       
modified predator_0 predator_1 ccm      x             2  5.102474e-01   
                                        y             2  3.262265e-01   
                               ccm_pval x             2  1.288860e-06   
                                        y             2  1.273575e-01   
                               granger  x             2  0.000000e+00   
                                        y             2  0.000000e+00   
         predator_1 predator_0 ccm      x             2  5.825572e-01   
                                        y             2  4.208703e-01   
                               ccm_pval x             2  1.575514e-22   
                                        y             2  6.220892e-02   
                               granger  x             2  0.000000e+00   
                                        y             2  0.000000e+00   
original predator_0 predator_1 ccm      x             2  6.278172e-01   
                                        y             2  3.429253e-01   
                               ccm_pval x             2  2.116752e-02   
                                        y             2  2.090055e-02   
                               granger  x             2  0.000000e+00   
                                        y             2  0.000000e+00   
         predator_1 predator_0 ccm      x             2  6.756005e-01   
                                        y             2  5.451641e-01   
                               ccm_pval x             2  3.467082e-19   
                                        y             2  1.825635e-06   
                               granger  x      

In [13]:
collated_df.groupby(['mode', 'agent_a', 'agent_b', 'test', 'dimension']).mean()

1000      2000  \
mode     agent_a    agent_b    test     dimension                       
modified predator_0 predator_1 ccm      x          0.022050  0.022050   
                                        y          0.028547  0.028547   
                               ccm_pval x          0.073043  0.073043   
                                        y          0.164269  0.164269   
                               granger  x          0.000000  0.000000   
                                        y          0.000000  0.000000   
         predator_1 predator_0 ccm      x          0.021994  0.021994   
                                        y          0.024176  0.024176   
                               ccm_pval x          0.184115  0.184115   
                                        y          0.219286  0.219286   
                               granger  x          0.000000  0.000000   
                                        y          0.000000  0.000000   
original predator_0 predator_1 ccm      x          0.046210  0.042654   
                                        y          0.065070  0.052416   
                               ccm_pval x          0.206359  0.222463   
                                        y          0.180169  0.268097   
                               granger  x          0.000000  0.000000   
                                        y          0.000000  0.000000   
         predator_1 predator_0 ccm      x          0.051178  0.057049   
                                        y          0.069767  0.067355   
                               ccm_pval x          0.151398  0.168662   
                                        y          0.174840  0.171406   
                               granger  x          0.000000  0.000000   
                                        y          0.000000  0.000000   

                                                       3000      4000  \
mode     agent_a    agent_b    test     dimension                       
modified predator_0 predator_1 ccm      x          0.022050  0.022050   
                                        y          0.028547  0.028547   
                               ccm_pval x          0.073043  0.073043   
                                        y          0.164269  0.164269   
                               granger  x          0.000000  0.000000   
                                        y          0.000000  0.000000   
         predator_1 predator_0 ccm      x          0.021994  0.021994   
                                        y          0.024176  0.024176   
                               ccm_pval x          0.184115  0.184115   
                                        y          0.219286  0.219286   
                               granger  x          0.000000  0.000000   
                                        y          0.000000  0.000000   
original predator_0 predator_1 ccm      x          0.034701  0.032478   
                                        y          0.032795  0.035946   
                               ccm_pval x          0.172663  0.168285   
                                        y          0.163243  0.212160   
                               granger  x          0.000000  0.000000   
                                        y          0.000000  0.000000   
         predator_1 predator_0 ccm      x          0.029287  0.030045   
                                        y          0.055199  0.041601   
                               ccm_pval x          0.242186  0.201083   
                                        y          0.224368  0.092323   
                               granger  x          0.000000  0.000000   
                                        y          0.000000  0.000000   

                                                       5000      6000  \
mode     agent_a    agent_b    test     dimension                       
modified predator_0 predator_1 ccm      x          0.022050  0.022050   
                                      